In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from joblib import dump


2024-07-18 09:57:10.190552: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 09:57:10.190654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 09:57:10.361821: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df=pd.read_csv("/kaggle/input/train-insurance/train_insurance.csv")
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [4]:
df.isnull().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder
lr=LabelEncoder()
cat_col=list(df.select_dtypes('object'))
for i in cat_col:
    df[i]=lr.fit_transform(df[i])
dump(lr, 'lr_encoder.pkl')

['lr_encoder.pkl']

In [6]:
df.corr()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
id,1.000000,-0.000031,-0.000088,-0.000045,0.000147,0.000217,0.000301,-0.000184,-0.000205,0.000550,-0.000182,0.000018
Gender,-0.000031,1.000000,0.157663,-0.018702,-0.000106,-0.087614,-0.117661,0.096989,0.010652,-0.116058,-0.009535,0.055212
Age,-0.000088,0.157663,1.000000,-0.078519,0.037041,-0.276248,-0.537738,0.287952,0.056327,-0.591443,-0.013293,0.122134
Driving_License,-0.000045,-0.018702,-0.078519,1.000000,-0.001329,0.013733,0.029308,-0.015563,-0.007300,0.042941,-0.000697,0.009197
Region_Code,0.000147,-0.000106,0.037041,-0.001329,1.000000,-0.022367,-0.023707,0.026468,-0.001741,-0.037606,-0.005537,0.012816
Previously_Insured,0.000217,-0.087614,-0.276248,0.013733,-0.022367,1.000000,0.191106,-0.836214,0.007665,0.236838,0.019431,-0.345930
Vehicle_Age,0.000301,-0.117661,-0.537738,0.029308,-0.023707,0.191106,1.000000,-0.189672,0.029708,0.395323,0.013875,-0.111630
Vehicle_Damage,-0.000184,0.096989,0.287952,-0.015563,0.026468,-0.836214,-0.189672,1.000000,0.012728,-0.241036,-0.019393,0.359780
Annual_Premium,-0.000205,0.010652,0.056327,-0.007300,-0.001741,0.007665,0.029708,0.012728,1.000000,-0.128559,0.003284,0.032261
Policy_Sales_Channel,0.000550,-0.116058,-0.591443,0.042941,-0.037606,0.236838,0.395323,-0.241036,-0.128559,1.000000,0.008999,-0.152733


In [7]:
x=df.iloc[:,1:-1].values
y=df.iloc[:,-1].values
print(x.shape,y.shape)

(11504798, 10) (11504798,)


In [8]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)
dump(sc, 'sc_scaler.pkl')

['sc_scaler.pkl']

In [9]:
ann=tf.keras.models.Sequential()
ann.add(tf.keras.layers.Input(shape=(10,)))
ann.add(tf.keras.layers.Dense(units=128,activation='relu'))
ann.add(tf.keras.layers.Dense(units=64,activation='relu'))
ann.add(tf.keras.layers.Dense(units=32,activation='relu'))
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = ann.fit(
    x, y,
    epochs=100,
    batch_size=1024,
    validation_split=0.2,
    callbacks=[checkpoint, early_stopping]
)

"ann=tf.keras.models.Sequential()\nann.add(tf.keras.layers.Input(shape=(10,)))\nann.add(tf.keras.layers.Dense(units=128,activation='relu'))\nann.add(tf.keras.layers.Dense(units=64,activation='relu'))\nann.add(tf.keras.layers.Dense(units=32,activation='relu'))\nann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))\nann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])\ncheckpoint = tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')\nearly_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)\nhistory = ann.fit(\n    x, y,\n    epochs=100,\n    batch_size=1024,\n    validation_split=0.2,\n    callbacks=[checkpoint, early_stopping]\n)"